In [ ]:
# install requests for API calls
!pip install requests


In [ ]:
#import requests library
import requests


In [2]:
# import Pandas library for CSV file handling
import pandas as pd


In [ ]:
GITHUB_TOKEN = ''     # replace the empty string with GitHub Token

In [ ]:
headers = {
    'Authorization': f'token {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

200 {'login': 'adnanksa', 'id': 43119179, 'node_id': 'MDQ6VXNlcjQzMTE5MTc5', 'avatar_url': 'https://avatars.githubusercontent.com/u/43119179?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/adnanksa', 'html_url': 'https://github.com/adnanksa', 'followers_url': 'https://api.github.com/users/adnanksa/followers', 'following_url': 'https://api.github.com/users/adnanksa/following{/other_user}', 'gists_url': 'https://api.github.com/users/adnanksa/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/adnanksa/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/adnanksa/subscriptions', 'organizations_url': 'https://api.github.com/users/adnanksa/orgs', 'repos_url': 'https://api.github.com/users/adnanksa/repos', 'events_url': 'https://api.github.com/users/adnanksa/events{/privacy}', 'received_events_url': 'https://api.github.com/users/adnanksa/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False, 'name': None, 'company': 

In [ ]:
# Initialize lists to store user
users_data = []

**Cleaning up company names as follows:**

---



1.   They are trimmed of whitespace
2.   Leading @ symbols are stripped
3.   They are converted to UPPERCASE


---

**Required user fields to fetch are as follows:**


---


1.   **login:** Their Github user ID
2.   **name:** Their full name
3.   **company:** The company they work at
4.   **location:** The city they are in
5.   **email:** Their email address
6.   **hireable:** Whether they are open to being hired
7.   **bio:** A short bio about them
8.   **public_repos:** The number of public repositories they have
9.   **followers:** The number of followers they have
10.  **following:** The number of people they are following
11.  **created_at:** When they joined Github


---

In [ ]:
def clean_company_name(company):
  """Clean up the company name as per the specified above rules."""
  if not company:
    return ""
  company = company.strip()        #Remove leading/trailing whitespace
  company = company.lstrip('@')    #Remove leading @ symbol
  return company.upper()           #Convert to UPPERCASE

In [ ]:
def fetch_users(city, min_followers=51):
  """Fetch users from GitHub API based on city with more than min_followers."""
  page = 1
  while True:
    url = f'https://api.github.com/search/users?q=location:{city}+followers:>={min_followers}&per_page=100&page={page}'
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
      print(f"Failed to fetch users: {response.status_code}")
      break

    data = response.json()
    items = data.get('items', [])

    if not items:
      break

    for user in items:
      #Fetch detailed user info from individual user endpoint
      user_detail = requests.get(user['url'], headers=headers).json()

      user_data = {
                'login': user_detail.get('login', ''),
                'name': user_detail.get('name', ''),
                'company': clean_company_name(user_detail.get('company', '')),
                'location': user_detail.get('location', ''),
                'email': user_detail.get('email', ''),
                'hireable': user_detail.get('hireable', ''),
                'bio': user_detail.get('bio', ''),
                'public_repos': user_detail.get('public_repos', 0),
                'followers': user_detail.get('followers', 0),
                'following': user_detail.get('following', 0),
                'created_at': user_detail.get('created_at', '')
            }
      users_data.append(user_data)

    page += 1

In [ ]:
# Fetch users in Mumbai with over 50 followers
fetch_users("Mumbai")

In [ ]:
# Save data to users.csv
users_df = pd.DataFrame(users_data)
users_df.to_csv('users.csv', index=False)

In [ ]:
import time

In [ ]:
# Load users data to get the list of users to query
users_df = pd.read_csv('users.csv')
repos_data = []

In [ ]:
def fetch_repos(username):
    """Fetch up to 500 most recently pushed repositories for a given user."""
    page = 1
    repos_fetched = 0
    while repos_fetched < 500:
        url = f'https://api.github.com/users/{username}/repos?per_page=100&page={page}&sort=pushed'
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch repos for {username}: {response.status_code}")
            break

        repos = response.json()
        if not repos:
            break

        for repo in repos:
            repo_data = {
                'login': username,
                'full_name': repo.get('full_name', ''),
                'created_at': repo.get('created_at', ''),
                'stargazers_count': repo.get('stargazers_count', 0),
                'watchers_count': repo.get('watchers_count', 0),
                'language': repo.get('language', ''),
                'has_projects': repo.get('has_projects', False),
                'has_wiki': repo.get('has_wiki', False),
                'license_name': repo.get('license', {}).get('key', '') if repo.get('license') else ''
            }
            repos_data.append(repo_data)
            repos_fetched += 1

            if repos_fetched >= 500:
                break

        page += 1
        time.sleep(1)  # To avoid hitting rate limits

In [ ]:
# Fetch repositories for each user in users.csv
for user in users_df['login']:
    print(f"Fetching repos for user: {user}")
    fetch_repos(user)

    # Save data incrementally after each user
    if repos_data:
        repos_df = pd.DataFrame(repos_data)
        repos_df.to_csv('repositories.csv', index=False)

print("Repository data fetching complete.")

Fetching repos for user: ValentineFernandes
Fetching repos for user: kovidgoyal
Fetching repos for user: slidenerd
Fetching repos for user: aryashah2k
Fetching repos for user: coding-parrot
Fetching repos for user: gkcs
Fetching repos for user: darshilparmar
Fetching repos for user: Kushal334
Fetching repos for user: ritz078
Fetching repos for user: PrasoonPratham
Fetching repos for user: rmehta
Fetching repos for user: LakshyaDuhoonISU
Fetching repos for user: dmalvia
Fetching repos for user: PiyushKumarSingh-90
Fetching repos for user: mfaisalkhatri
Fetching repos for user: omsandippatil
Fetching repos for user: anujvyas
Fetching repos for user: jalajthanaki
Fetching repos for user: alisolanki
Fetching repos for user: ForceGT
Fetching repos for user: Sahil4883
Fetching repos for user: yTakkar
Fetching repos for user: saineshwar
Fetching repos for user: codescientists
Fetching repos for user: amangoeliitb
Fetching repos for user: debugger22
Fetching repos for user: shifa123
Fetching r

# Questions based on the data sets fetched from GitHub API



---



1.   *Who are the top 5 users in **Mumbai** with the highest number of followers? List their **login** in order, comma-separated.*



---




In [3]:
# Load users.csv from my Github repository
users_url = 'https://raw.githubusercontent.com/22f3001673/TDS-Project1/main/users.csv'
users_df = pd.read_csv(users_url)


In [ ]:
# Sort users by the 'followers' column in descending order and select the top 5
top_users = users_df.sort_values(by='followers', ascending=False).head(5)

In [ ]:
# Get the 'login' column as a comma-separated string
top_users_logins = ', '.join(top_users['login'])
print(top_users_logins)

ValentineFernandes, kovidgoyal, slidenerd, aryashah2k, coding-parrot




---


2.   *Who are the 5 earliest registered GitHub users in **Mumbai**? List their **login** in ascending order of **created_at**, comma-separated.*



---



In [ ]:
# Convert the 'created_at' column to datetime format
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

In [ ]:
# Sort users by 'created_at' in ascending order
sorted_users = users_df.sort_values(by='created_at', ascending=True)

In [ ]:
# Select the top 5 earliest registered users
top_5_earliest_users = sorted_users['login'].head(5)

In [ ]:
# Output their logins as a comma-separated string
print(','.join(top_5_earliest_users))

ivank,sandeepshetty,svs,nitinhayaran,nischal





---


*3.   What are the 3 most popular license among these users? Ignore missing licenses. List the **license_name** in order, comma-separated.*



---



In [12]:
# Load repositories.csv from my Github repository
repositories_url = 'https://raw.githubusercontent.com/22f3001673/TDS-Project1/main/repositories.csv'
repos_df = pd.read_csv(repositories_url)

In [ ]:
# Filter out rows where 'license_name' is missing
repos_with_license = repos_df[repos_df['license_name'].notna() & (repos_df['license_name'] != '')]

In [ ]:
# Count occurrences of each license and select the top 3
top_licenses = repos_with_license['license_name'].value_counts().head(3)

In [ ]:
# Output the license names as a comma-separated string
print(','.join(top_licenses.index))

mit,apache-2.0,other






---


*4.   Which **company** do the majority of these developers work at?*



---



In [ ]:
most_common_company = users_df['company'].value_counts().idxmax()

In [ ]:
# Output the most common company
print(most_common_company)

MASAI SCHOOL




*5.   Which **programming language** is most popular among these users?*


In [ ]:
# Filter out rows where 'language' is missing or empty
repos_with_language = repos_df[repos_df['language'].notna() & (repos_df['language'] != '')]

In [ ]:
# Count occurrences of each language and select the most common one
most_popular_language = repos_with_language['language'].value_counts().idxmax()

In [ ]:
# Output the most popular language
print(most_popular_language)

JavaScript





---


*6.   Which **programming language** is the second most popular among users who joined after 2020?*


---




In [ ]:
# Convert 'created_at' to datetime
users_df['created_at'] = pd.to_datetime(users_df['created_at'])

In [ ]:
# Filter for users who joined after 2020
users_after_2020 = users_df[users_df['created_at'].dt.year > 2020]

In [ ]:
# Load the repositories.csv file
repos_df = pd.read_csv('repositories.csv')

In [ ]:
# Filter repositories to include only those owned by users who joined after 2020
filtered_repos = repos_df[repos_df['login'].isin(users_after_2020['login'])]

In [ ]:
# Filter out rows where 'language' is missing or empty
repos_with_language = filtered_repos[filtered_repos['language'].notna() & (filtered_repos['language'] != '')]

In [ ]:
# Count occurrences of each language
language_counts = repos_with_language['language'].value_counts()

In [ ]:
# Get the second most popular language
second_most_popular_language = language_counts.index[1] if len(language_counts) > 1 else None

In [ ]:
# Output the second most popular language
print(second_most_popular_language)

HTML





---


*7.   Which **languag**e has the highest average number of stars per repository?*


---



In [ ]:
# Filter out rows where 'language' is missing or empty
repos_with_language = repos_df[repos_df['language'].notna() & (repos_df['language'] != '')]

In [ ]:
# Group by 'language' and calculate the average 'stargazers_count' for each language
average_stars_per_language = repos_with_language.groupby('language')['stargazers_count'].mean()

In [ ]:
# Find the language with the highest average number of stars
language_with_highest_avg_stars = average_stars_per_language.idxmax()
highest_avg_stars = average_stars_per_language.max()

In [ ]:
# Output the language with the highest average stars and the average star count
print(f"{language_with_highest_avg_stars} has the highest average stars per repository: {highest_avg_stars}")

TSQL has the highest average stars per repository: 571.0





---


*8.   Let's define **leader_strength** as **followers / (1 + following)**. Who are the top 5 in terms of **leader_strength**? List their **login** in order, comma-separated.*



---



In [ ]:
# Calculate leader_strength
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

In [ ]:
# Sort by leader_strength in descending order and get the top 5 logins
top_5_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)['login'].tolist()

In [ ]:
# Output the top 5 logins as a comma-separated string
print(", ".join(top_5_leaders))

kovidgoyal, coding-parrot, gkcs, slidenerd, dmalvia





---



*9.   What is the **correlation** between the number of followers and the number of public repositories among users in **Mumbai**?*



---



In [ ]:
correlation = users_df['followers'].corr(users_df['public_repos'])
# Output the correlation rounded to 3 decimal places
print(f"Correlation between no. of followers and public repositories: {correlation:.3f}")

Correlation between no. of followers and public repositories: 0.034





---


*10.   Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.*


---



In [ ]:
import statsmodels.api as sm

In [ ]:
# Define predictor and response variables
X = users_df['public_repos']
y = users_df['followers']

In [ ]:
# Add a constant to the predictor (intercept term)
X = sm.add_constant(X)

In [ ]:
# Fit the regression model
model = sm.OLS(y, X).fit()


In [ ]:
# Get the slope of the regression (coefficient of public_repos)
slope = model.params['public_repos']
print(f"Regression slope of followers on repos: {slope:.3f}")

Regression slope of followers on repos: 0.100




---



*11.   Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?*


---



In [13]:
correlation_pro_wiki = repos_df['has_projects'].astype(int).corr(repos_df['has_wiki'].astype(int))
# Output the correlation rounded to 3 decimal places
print(f"Correlation between projects and wiki enabled: {correlation_pro_wiki:.3f}")

Correlation between projects and wiki enabled: 0.160





---


*12.   Do hireable users follow more people than those who are not hireable?*



---



In [ ]:
# Replace NaN values in 'hireable' with False and ensure it's a boolean type
users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)

<ipython-input-87-3a01ae0099db>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)


In [ ]:
# Calculate the average 'following' count for hireable and non-hireable users
average_following_hireable = users_df[users_df['hireable'] == True]['following'].mean()
average_following_not_hireable = users_df[users_df['hireable'] == False]['following'].mean()

In [ ]:
# Calculate the difference and round to 3 decimal places
difference = round(average_following_hireable - average_following_not_hireable, 3)

In [ ]:
# Output the difference
print(f"Difference in average following (hireable - non-hireable): {difference}")

Difference in average following (hireable - non-hireable): 8.175




---



*13.   Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)*


---




In [15]:
# Filter users with bios and create a new DataFrame to avoid chained assignment issues
users_with_bios = users_df[users_df['bio'].notna() & (users_df['bio'] != '')].copy()

In [16]:
# Calculate the length of each bio in Unicode words
users_with_bios.loc[:, 'bio_length'] = users_with_bios['bio'].apply(lambda x: len(x.split()))

In [17]:
# Calculate the correlation between bio length and followers
correlation_bio_followers = users_with_bios['bio_length'].corr(users_with_bios['followers'])

In [18]:
# Print the correlation rounded to 3 decimal places
print(f"Correlation between bio length and followers: {correlation_bio_followers:.3f}")

Correlation between bio length and followers: -0.008




---



*14.   Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated*


---




In [ ]:
# Convert 'created_at' to datetime in UTC to avoid time zone issues
repos_df['created_at'] = pd.to_datetime(repos_df['created_at'], utc=True)

In [ ]:
# Filter for repositories created on weekends (Saturday = 5, Sunday = 6)
weekend_repos = repos_df[repos_df['created_at'].dt.weekday.isin([5, 6])]

In [ ]:
# Count repositories created on weekends for each user
weekend_repos_count = weekend_repos['login'].value_counts()

In [ ]:
# Get the top 5 users by weekend repository count
top_5_users = weekend_repos_count.head(5).index.tolist()

In [ ]:
# Output the top 5 users' logins as a comma-separated string
print(','.join(top_5_users))

Kushal334,alokproc,vinod1988,patilswapnilv,rajeshpillai





---


*15.   Do people who are hireable share their email addresses more often?
[fraction of users with email when hireable=true] minus [fraction of users with email for the rest] (to 3 decimal places, e.g. 0.123 or -0.123)*



---



In [5]:
# Fill missing 'hireable' values with False (assuming NaN means non-hireable)
users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)

# Separate hireable and non-hireable users
hireable_users = users_df[users_df['hireable'] == True]
non_hireable_users = users_df[users_df['hireable'] == False]

# Calculate fraction with email for both groups, checking if groups are non-empty
hireable_email_fraction = hireable_users['email'].notna().mean() if not hireable_users.empty else 0
non_hireable_email_fraction = non_hireable_users['email'].notna().mean() if not non_hireable_users.empty else 0

# Compute the difference
difference_email_fraction = round(hireable_email_fraction - non_hireable_email_fraction, 3)
print(difference_email_fraction)

0.222


<ipython-input-5-795c2c9a3b93>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_df['hireable'] = users_df['hireable'].fillna(False).astype(bool)





---


*16.   Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)*



---



In [ ]:
# Filter out users with missing or empty 'name'
users_with_name = users_df[users_df['name'].notna() & (users_df['name'] != '')]

In [ ]:
# Extract the last word from 'name' as the surname
users_with_name['surname'] = users_with_name['name'].str.strip().str.split().str[-1]

<ipython-input-93-96501fa48f10>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_name['surname'] = users_with_name['name'].str.strip().str.split().str[-1]


In [ ]:
# Count occurrences of each surname
surname_counts = users_with_name['surname'].value_counts()

In [ ]:
# Find the maximum count
max_count = surname_counts.max()

In [ ]:
# Get all surnames with the maximum count
most_common_surnames = surname_counts[surname_counts == max_count].index.tolist()

In [ ]:
# Sort alphabetically if there's a tie and output as comma-separated string
most_common_surnames.sort()
print(','.join(most_common_surnames))

Singh
